# Associação entre variáveis qualitativas

É mais comum falar sobre a associação ou relação (correlação) entre variáveis quantitativas, porém é possível identificar associações entre variáveis qualitativas. Para essa última, existem medidas que são úteis para identificar a associação e sua intensidade (ou o grau de dependência entre elas) de modo que seja possível saber melhor os seus resultados com base em outras váriáveis. Para isso é necessário calcular a distribuição conjunta entre as variáveis por meio da tabela de contingência.

A princiais medidas de associação entre variáveis qualitativas são:

1) Coeficiente de contingência;

2) Coeficiente de contingência modificado;

3) Coeficiente *V* de *Crammer*.

## Importando as bibliotecas

In [1]:
import pandas as pd
from scipy.stats import chi2_contingency

## Importando a base de dados

In [3]:
casa = pd.read_csv('Mumbai1.csv')

Criando as tabelas de contingência com e sem as margens.

Calcularei a tabela de contingência para as variáveis *Swimming Pool* (se a casa possui piscina ou não) e *New/Resale* (se a casa é nova ou está para revenda). Não importa a ordem das variáveis, pois o resultado dos coeficientes será o mesmo.

A tabela de cotingência possui dimensões *k* x *r*, onde *k* é o número de linha e *r* o número de colunas da tabela. Para esse exemplo, temos cada variável com 2 classes, logo a tabela será 2x2.


In [8]:
tabela = pd.crosstab(casa['Swimming Pool'],casa['New/Resale'])
tabela2 =  pd.crosstab(casa['Swimming Pool'],casa['New/Resale'], margins=True)

Abaixo as tabelas calculadas.

In [9]:
#tabela sem as margens
tabela

New/Resale,0,1
Swimming Pool,,
0,2678,760
1,1500,1409


In [10]:
#tabela com as margens
tabela2

New/Resale,0,1,All
Swimming Pool,,,
0,2678,760,3438
1,1500,1409,2909
All,4178,2169,6347


## Estatística $\chi^2$

Para calcular os coeficientes é necessário calcular a estatística $\chi^2$. A forma para calcular essas estatística é utilizando a **tabela de contingência original** e uma outra **tabela de contingência esperada** (calculada assumindo que as variáveis são independentes).

A forma de cálculo é dada abaixo

$\chi^2 = \sum_{i=1}^{r} \sum_{j=1}^k \dfrac{(O_{ij} - E_{ij})^2}{E_{ij}}$

Podem haver casos que o número de classe das variáveis seja muito grande, tornando o cálculo inviável. Porém, abaixo, a função *chi2_contingency* permite calcular o que é necessário: o estatística $\chi^2$, o *p-valor* da estatística, o grau de liberdade e a tabela de contingência esperada.

In [25]:
print('Estatística Chi² :',chi2_contingency(tabela)[0])
print('p-valor :', chi2_contingency(tabela)[1])
print('Graus de liberdade :',chi2_contingency(tabela)[2])
print(''*254)
print(pd.DataFrame(chi2_contingency(tabela)[3]))

Estatística Chi² : 484.44541695318026
p-valor : 2.3037749668207414e-107
Graus de liberdade : 1

             0            1
0  2263.110761  1174.889239
1  1914.889239   994.110761


Um valor grande da estatística $\chi^2$ indica uma associação entre as variáveis, que parece ser o nosso caso.

## Coeficiente de contingência *C*

A medida mais conhecida para medir a associação entre variáveis qualitativas é o coeficiente de contingência *C*. Esse coeficiente pode ser obtido por meio da expressão logo abaixo

$C = \sqrt{\dfrac{\chi^2}{n + \chi^2}}$


onde $n$ é o número de observações da amostra.

**Calculando o coeficiente de contingência**

In [13]:
(chi2_contingency(tabela)[0]/(chi2_contingency(tabela)[0]+tabela2['All']['All']))**0.5

0.2662969084029654

### Teste de significância do coeficiente de contingência

Depois de encontrar o valor do coeficiente de contingênci, o próximo passo é testar se a medida de associação é estatisticamente significativa. Testa-se a hipótese de nulidade de que não há associação, ou seja, que o valor observado ocorre aleatoriamente entre as amostras, mesmo que as populações não apresentem relação alguma. Para saber se o coeficiente *C* é estatisticamente significativo, basta que a estatística $\chi^2$ seja estatisticamente significativa.

Hipóteses:

$H_0$ : Não há associação entre as duas variáveis

$H_1$ : Há associação entre as duas variáveis

Se o *p-valor* de $\chi^2$ for inferior ao nível de significância escolhido (por padrão 5%), então a rejeita-se a hipótse de nulidade de associação.

O resultado do *p-valor* calculado foi abaixo do valor do nível de significância, logo concluímos que há associação entre as duas variáveis qualitativas *Swimming Pool* e *ew/Resale*.

In [36]:
print('p-valor :', chi2_contingency(tabela)[1])

p-valor : 2.3037749668207414e-107


### Limitações do coeficiente de contingência *C*

Para coeficientes de associação:

1) houver ausência de associação, então o valor do coeficiente é zero;

2) Se a associação for perfeita, então o valor do coeficiente será 1.

O coeficiente *C* é sempre menor que 1, então mesmo que as variáveis tenham uma associação perfeita, o valor do coeficiente nuna será 1.

No exemplo abaixo eu inseri a mesma variável *Swimming Pool* que é perfeitamente associada consigo mesma e o valor do coeficente de contingência *C* não é só menor que 1, mas também está muito abaixo desse valor.

In [33]:
tabela3 = pd.crosstab(casa['Swimming Pool'],casa['Swimming Pool'])
tabela4 = pd.crosstab(casa['Swimming Pool'],casa['Swimming Pool'], margins=True)
#coeficiente de contingência
(chi2_contingency(tabela3)[0]/(chi2_contingency(tabela3)[0]+tabela4['All']['All']))**0.5

0.7069945671266584

### Coeficiente de contingência *C* modificado

Dada a inconveniência do coeficiente *C* é possível modificá-lo de duas formas.

A primeira forma pode ser vista abaixo, onde *k* é o menor número entre o número de linhas e de colunas da tabela de contingência.

$C^* = \sqrt{\dfrac{\chi^2}{n + \chi^2}} \cdot \sqrt{\dfrac{min(k-1, r-1)}{min(k-1, r-1)-1}}$

In [41]:
k=2
r=2
((chi2_contingency(tabela3)[0]/(chi2_contingency(tabela3)[0]+tabela4['All']['All']))**0.5)*((k/(k-1))**0.5)

0.9998413053546159

Com a modificação o valor ainda é menor que 1 (associação perfeita), porém está bem mais próximo de 1.

A segunda forma é descrita por *Morettin & Bussab* (2010) onde o coeficiente é chamado de **T** e sua fórmula de cálculo é descrita abaixo

$T = \sqrt{\dfrac{\chi^2/n}{(k-1) \cdot (r-1)}}$

Onde *k* e *r* são o número de colunas e linhas da tabela de contingência.

E conforme abaixo ficou bem próximo de 1 e um pouco diferente do resultado anterior.

In [50]:
((chi2_contingency(tabela3)[0]/tabela4['All']['All'])/((k-1)*(r-1)))**0.5

0.9996826862372318

Calculando os coeficientes de contingência modificados para associação entre *Swimming Pool* e *New/Resale*.

Para os dois resultados obtivemos uma associação fraca entre essas variáveis.

In [49]:
print(((chi2_contingency(tabela)[0]/(chi2_contingency(tabela)[0]+
                               tabela2['All']['All']))**0.5)*((k/(k-1))**0.5))
print(((chi2_contingency(tabela)[0]/tabela2['All']['All'])/((k-1)*(r-1)))**0.5)

0.3766006994814995
0.2762728301702427


Outras **limitações** do coeficiente de contingência:

- ele não pode ser comparado com nenhuma outra medida de associação, como os coeficientes de correlação (de *Pearson*, *Spearman* e *Kendall*).

- é influenciado pelo número de linhas e de colunas da tabela de contingência.

## Coeficiente *V* de *Crámer*

É uma alternativa ao coeficiente de contingência *C* que é dado pela fórmula abaixo

$V = \sqrt{\dfrac{\chi^2}{n \cdot min(k-1, r-1)}}$

A vantagem desse coeficiente é superar a inconveniência da influência do número de linhas e colunas sobre o cálculo da medida de associação.  O valor 0 corresponde a ausência de associação , valores próximos de zero correspondem a fraca associação e valores mais próximos de 1 correspondem a associaçãao mais forte.

Calculando o coeficiente *V* de *Crámer*.


In [52]:
((chi2_contingency(tabela)[0])/(tabela2['All']['All']*(k-1)))**0.5

0.2762728301702427

O valor do coeficiente *V* ficou bem próximo dos valores dos outros coeficientes calculados anteriormente, revelando uma associação fraca entre as duas variáveis.